# **Phase 1: Study of Classical Statistical Models for Demand Forecasting**

In this initial project phase, we focus on **evaluating forecasting models based on classical statistical methods**, aiming to establish a baseline for comparison with more complex models (e.g., machine learning, deep learning) to be developed later.

## Main Objective:
Optimize inventory and purchasing management, targeting a **20% reduction in overstocking within 6 months**.

## Target Variables:
- **Demand**: `Sales_Volume`
- **Inventory**: `Stock_Quantity`

## Implemented Statistical Models:
- `HoltWinters`
- `SeasonalNaive`
- `DynamicOptimizedTheta`
- `HistoricAverage` (fallback model)
- `AutoARIMA`
- `AutoETS`
- `AutoCES`
- `AutoTheta`

## Evaluation Metrics:
- **RMSE** (Root Mean Squared Error)
- **MAE** (Mean Absolute Error)

## Approach:
- Utilization of the `StatsForecast` library for model training and evaluation.
- Logarithmic transformation of target variables to stabilize variance.
- Forecasts generated for a **28-day horizon**.
- Cross-validation and performance comparison among models.

This phase helps identify which statistical methods are most suitable for the given time series, serving as a reference for future iterations with more advanced techniques.

## Import Libraries

In [1]:
# Standard Libraries 
import pandas as pd
import numpy as np
import os
import plotly.express as px
import joblib

# Specialized Libraries
from statsforecast import StatsForecast
from statsforecast.models import (
    HoltWinters, HistoricAverage, DynamicOptimizedTheta as DOT,
    SeasonalNaive, AutoARIMA, AutoETS, AutoCES, AutoTheta
)
from utilsforecast.losses import mae, rmse

# Custom Libraries
from smart_supply_chain_ai.utils.metrics import evaluate_cv, get_best_model_forecast

# Notebook mlflow Loggings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="pkg_resources is deprecated")
warnings.filterwarnings("ignore", category=FutureWarning)
pd.set_option('display.max_columns', None)

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


## Load Data

In [2]:
# Define data paths
data_path = os.path.join('../data', 'processed')
docs_path = os.path.join('../docs/')
path_models = os.path.join('../models/')

saved_models = os.listdir(path_models)

In [3]:
# Create a list of models and instantiation parameters
models = [
    HoltWinters(season_length=7),
    SeasonalNaive(season_length=7),
    DOT(season_length=7)
]

In [4]:
# Instantiate StatsForecast class as sf
sf = StatsForecast( 
    models=models,
    freq='D', 
    fallback_model = HistoricAverage(),
    n_jobs=-1,
)

In [5]:
# Load Pickle file
compare_data = pd.read_parquet(data_path + '/data_for_compare.parquet')
read_data = pd.read_parquet(data_path + '/data_for_train.parquet')

In [6]:
# View shape
read_data.shape, compare_data.shape

((70719, 33), (8736, 33))

In [7]:
# Duplicated data
read_data.duplicated().sum()

0

In [8]:
# Count the number of duplicate product entries received on the same date
read_data.duplicated(subset=['received_date', 'product_id']).sum()

5390

# Prepare data for time series analysis

In [9]:
# Create a copy of the original DataFrame to preserve the raw data
df = read_data.copy()

In [10]:
# Display the first five rows of the DataFrame to preview the data
df.head()


,received_date,lpo,in_season,product,product_id,category,sub_category,shelf_life_days,maximum_days_on_sale,unit_of_measurement,supplier_rating,supplier,supplier_id,distance_km,moq,storage_recommendation,temperature_classification,precipitation_classification,wind_classification,weather_severity,day_classification,is_holiday,is_weekend,sales_demand,sales_volume,lead_time,min_stock,max_stock,stock_quantity,delivery_lag,expiration_status,inventory_turnover_rate,doi_inventory_turnover
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2022-12-09,2022-11-29,True,Lime,1019979|P,Fresh Foods,Fruits,21.0,7.0,lb,5.0,Lemon & Lime Co.,1536622|S,190.0,35.0,Refrigerated,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Weekdays,False,False,Normal,526,7,3520,4224,5275,10,Nearing,21.494078,47.0
1,2022-12-09,2022-12-01,False,White Bread,1033235|P,Bakery,Bread,5.0,2.0,unit,3.0,Bakery Fresh Co.,1644445|S,45.0,75.0,Room Temperature,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Weekdays,False,False,Normal,688,3,2271,3028,2436,8,Expired,35.607450,28.0
2,2022-12-09,2022-12-07,False,Sardines,1093592|P,Fresh Foods,Seafood,2.0,1.0,lb,2.0,OceanHarvest Seafood,1168079|S,180.0,40.0,Refrigerated,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Weekdays,False,False,Normal,33,5,155,186,155,2,Nearing,22.882402,44.0
3,2022-12-09,2022-11-29,False,Avocado,1113134|P,Fresh Foods,Fruits,7.0,3.0,unit,4.0,Avocado Avenue,1840542|S,280.0,30.0,Room Temperature,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Weekdays,False,False,Normal,718,7,5975,7170,6677,10,Expired,42.740888,23.0
4,2022-12-09,2022-12-03,False,Cod,1119767|P,Fresh Foods,Seafood,2.0,1.0,lb,1.0,OceanHarvest Seafood,1168079|S,180.0,40.0,Refrigerated,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Weekdays,False,False,Normal,45,6,150,180,156,9,Expired,44.152735,23.0


In [11]:
# Create columns ordination
order_columns = df.columns.tolist()

In [12]:
# Select columns for dtype 
number_max = ['delivery_lag', 'lead_time', 'max_stock']
number_mean = list(set(df.select_dtypes(include=[np.number]).columns.tolist()) - set(number_max) - set(['sales_volume', 'stock_quantity', 'min_stock'])) 
not_numbers = list(set(df.select_dtypes(exclude=[np.number]).columns.tolist()) - set(['received_date', 'product_id']))

In [13]:
# Create rules for groupby
agg_rules = {
    'sales_volume': 'sum',
    'stock_quantity': 'last',
    'min_stock': 'min',
    'supplier_id': 'last',
}

for col in number_max:
    agg_rules[col] = 'max'

for col in number_mean:
    agg_rules[col] = 'mean'

for col in not_numbers:
    agg_rules[col] = 'last'

In [14]:
# Group duplicated data
df = df.groupby(['received_date', 'product_id'], as_index=False).agg(agg_rules).reset_index(drop=True)

In [15]:
# Confirm duplicates
df.duplicated(subset=['received_date', 'product_id'], keep=False).sum()

0

In [16]:
# Log transform
df['stock_log'] = np.log1p(df['stock_quantity'])
df['sales_log'] = np.log1p(df['sales_volume'])

# Model's

In [17]:
# Create a list of models and instantiation parameters
models = [
    HoltWinters(season_length=7),
    SeasonalNaive(season_length=7),
    DOT(season_length=7)
]

In [18]:
# Instantiate StatsForecast class as sf
sf = StatsForecast( 
    models=models,
    freq='D', 
    fallback_model = HistoricAverage(),
    n_jobs=-1,
)

# Stock Quantity

In [19]:
# Rename columns to fit the StatsForecast requirements
df_stock = df.rename(columns={'received_date': 'ds', 'product_id': 'unique_id', 'stock_log': 'y'})

# Simplify the DataFrame to include only necessary columns
df_stock_simple = df_stock[['unique_id', 'ds', 'y']].copy()

In [20]:
df_stock_simple.head()

,unique_id,ds,y
0,1019979|P,2022-12-09,8.570924
1,1033235|P,2022-12-09,7.798523
2,1093592|P,2022-12-09,5.049856
3,1113134|P,2022-12-09,8.806574
4,1119767|P,2022-12-09,5.056246


## Train multiple models for many series

In [21]:
# Check if the multiple models StatsForecast for stock is already saved
if 'stock_multiple_models.joblib' in saved_models:
    # Load the saved multiple models StatsForecast for stock
    forecasts_df = joblib.load(path_models + 'stock_multiple_models.joblib')
else:
    # Generate forecasts for the next 28 days with 95% prediction intervals
    forecasts_df = sf.forecast(df=df_stock_simple, h=28, level=[90])

    # Save the multiple models StatsForecast for stock
    StatsForecast.save(forecasts_df, path_models + 'stock_multiple_models.joblib')


In [22]:
# Inverse transformation of log1p to original scale using expm1
forecasts_df[forecasts_df.select_dtypes(include=[np.number]).columns.to_list()] = np.expm1(forecasts_df.select_dtypes(include=[np.number]))

In [23]:
# Display the first 14 rows of the forecasts DataFrame
forecasts_df.head(14)

,unique_id,ds,HoltWinters,HoltWinters-lo-90,HoltWinters-hi-90,SeasonalNaive,SeasonalNaive-lo-90,SeasonalNaive-hi-90,DynamicOptimizedTheta,DynamicOptimizedTheta-lo-90,DynamicOptimizedTheta-hi-90
0,1003530|P,2025-05-10,401.652434,110.905128,1447.807445,523.0,89.381558,3036.964901,405.043207,123.443046,1803.626994
1,1003530|P,2025-05-11,324.835498,89.556168,1171.407950,568.0,97.143333,3297.858833,405.043207,102.983569,1242.142859
2,1003530|P,2025-05-12,355.573514,98.098866,1282.008330,564.0,96.453397,3274.668261,405.043207,78.484393,1413.497595
3,1003530|P,2025-05-13,420.441401,116.126924,1515.413542,574.0,98.178236,3332.644691,405.043207,103.817696,1238.475997
4,1003530|P,2025-05-14,443.757596,122.606931,1599.309289,468.0,79.894944,2718.094539,405.043207,128.987141,1629.707132
5,1003530|P,2025-05-15,350.128889,96.585632,1262.418546,476.0,81.274815,2764.475683,405.043207,115.952438,1481.365202
6,1003530|P,2025-05-16,401.764005,110.936001,1448.210644,545.0,93.176203,3164.513045,405.043207,100.426470,1585.053758
7,1003530|P,2025-05-17,401.491179,110.860118,1447.229739,523.0,42.644620,6290.176364,405.043207,105.290541,1610.105871
8,1003530|P,2025-05-18,324.705006,89.519688,1170.941191,568.0,46.392726,6830.449144,405.043207,99.462554,1587.122385
9,1003530|P,2025-05-19,355.430712,98.058872,1281.498478,564.0,46.059561,6782.424897,405.043207,112.675708,1338.797423


In [24]:
# Reverter a transformação log1p para os valores originais
revert_log = np.expm1(df_stock_simple['y'])
df_stock_simple.loc[: , 'y_origin'] = revert_log

In [25]:
# Filter the DataFrame for dates after June 1, 2024
plot_df = df_stock_simple.loc[df_stock_simple['ds'] > '2024-06-01'].copy()
plot_df.drop(columns=['y'], inplace=True)
plot_df.rename(columns={'y_origin': 'y'}, inplace=True)

In [26]:
# Plot forecasts for the specified date range
sf.plot(plot_df, forecasts_df, engine='plotly')

## Evaluate the model’s performance

In [27]:
df_stock_simple.head()

,unique_id,ds,y,y_origin
0,1019979|P,2022-12-09,8.570924,5275.0
1,1033235|P,2022-12-09,7.798523,2436.0
2,1093592|P,2022-12-09,5.049856,155.0
3,1113134|P,2022-12-09,8.806574,6677.0
4,1119767|P,2022-12-09,5.056246,156.0


In [28]:
if 'stock_cv.joblib' in saved_models:
    # Load the saved cross-validation results for stock
    stock_cv_df = joblib.load(path_models + 'stock_cv.joblib')
else:
    # Perform cross-validation to evaluate model performance
    stock_cv_df = sf.cross_validation(
        df=df_stock_simple.drop(columns=['y_origin']),
        h=28,
        step_size=28,
        n_windows=2
    )

    # Save the cross-validation results for stock
    StatsForecast.save(stock_cv_df, path_models + 'stock_cv.joblib')

In [29]:
# Display the first rows of the cross-validation DataFrame
stock_cv_df.head()

,unique_id,ds,cutoff,y,HoltWinters,SeasonalNaive,DynamicOptimizedTheta
0,1003530|P,2025-01-01,2024-12-30,6.171701,6.029430,6.507278,6.019058
1,1003530|P,2025-01-04,2024-12-30,6.621406,5.725228,6.284134,6.019058
2,1003530|P,2025-01-06,2024-12-30,6.487684,5.783928,6.361302,6.019058
3,1003530|P,2025-01-09,2024-12-30,6.621406,5.927278,6.331502,6.019058
4,1003530|P,2025-01-10,2024-12-30,6.469250,6.113249,6.190315,6.019058


In [30]:
# Evaluate model performance using Mean Absolute Error (MAE)
evaluation_mae_stock = evaluate_cv(stock_cv_df, mae)
evaluation_mae_stock.head()

,unique_id,HoltWinters,SeasonalNaive,DynamicOptimizedTheta,best_model
0,1003530|P,0.651370,0.482786,0.554629,SeasonalNaive
1,1007004|P,0.608582,0.564497,0.580273,SeasonalNaive
2,1009699|P,0.361535,0.324640,0.260752,DynamicOptimizedTheta
3,1017723|P,0.608455,0.814891,0.638863,HoltWinters
4,1018159|P,0.666621,0.649316,0.597169,DynamicOptimizedTheta


In [31]:
# Evaluate model performance using Root Mean Square Error (RMSE)
evaluation_rmse_stock = evaluate_cv(stock_cv_df, rmse)
evaluation_rmse_stock.head()

,unique_id,HoltWinters,SeasonalNaive,DynamicOptimizedTheta,best_model
0,1003530|P,0.925749,0.925140,0.888410,DynamicOptimizedTheta
1,1007004|P,0.929623,0.983055,0.939472,HoltWinters
2,1009699|P,0.610802,0.694768,0.590499,DynamicOptimizedTheta
3,1017723|P,1.006542,1.240259,1.023053,HoltWinters
4,1018159|P,1.039955,1.171942,1.080194,HoltWinters


In [32]:
# Display the best model counts based on MAE evaluation
evaluation_mae_stock['best_model'].value_counts().to_frame().reset_index()

,best_model,count
0,HoltWinters,80
1,DynamicOptimizedTheta,73
2,SeasonalNaive,17


In [33]:
# Display the best model counts based on RMSE evaluation
evaluation_rmse_stock['best_model'].value_counts().to_frame().reset_index()

,best_model,count
0,HoltWinters,97
1,DynamicOptimizedTheta,72
2,SeasonalNaive,1


In [34]:
seasonal_ids = evaluation_rmse_stock.query('best_model == "HoltWinters"')['unique_id']
sf.plot(plot_df, forecasts_df, unique_ids=seasonal_ids, models=["HoltWinters","DynamicOptimizedTheta"], engine='plotly')

## Select the Best Model

In [35]:
# Prepare data for plotting
prod_forecasts_df = get_best_model_forecast(forecasts_df, evaluation_mae_stock)
prod_forecasts_df.head()

,unique_id,ds,best_model,best_model-lo-90,best_model-hi-90
0,1003530|P,2025-05-10,523.0,89.381558,3036.964901
1,1003530|P,2025-05-11,568.0,97.143333,3297.858833
2,1003530|P,2025-05-12,564.0,96.453397,3274.668261
3,1003530|P,2025-05-13,574.0,98.178236,3332.644691
4,1003530|P,2025-05-14,468.0,79.894944,2718.094539


In [36]:
# Plot to unique_ids and some selected models
sf.plot(plot_df, prod_forecasts_df, level=[90], engine='plotly')

## Technical Summary of Time Series Model Analysis

### Objective
Compare the performance of **HoltWinters (HW)**, **SeasonalNaive (SN)**, and **DynamicOptimizedTheta (DOT)** models on daily time series with weekly seasonality, using error metrics (MAE and RMSE) to identify the most suitable model.

---

### Model Performance

| Metric | Best Model | Frequency |
|--------|-----------------------------|------------|
| MAE    | HW (80), DOT (73), SN (17)  |    Dialy
| RMSE   | HW (97), DOT (72), SN (1)   |    Dialy

---

### Forecast Visualization Insights

- Model forecasts are often **flat** (constant), with **wide confidence intervals**, revealing **high volatility** and **uncertainty** in the series.
- This suggests that the tested models struggle with the **noise** present in the data.

---

### Technical Recommendations

1. **Model Selection:**
   - Use **DOT** to minimize average daily errors.
   - Prefer **HW** to avoid large deviations.

2. **Suggested Improvements:**
   - **Add exogenous variables** (holidays, promotions, etc.).
   - **Handle outliers** to reduce RMSE impact.
   - **Reassess seasonality**, testing models with monthly cycles or without explicit seasonality.

---

##  Automatic Models for Stock Quantity

In [37]:
# Define parameters
season_length=7
horizon=28

In [38]:
# Automatic Models
auto_models = [
    AutoARIMA(season_length=season_length),
    AutoETS(season_length=season_length),
    AutoCES(season_length=season_length),
    AutoTheta(season_length=season_length)
    ]

In [39]:
# Instantiate StatsForecast class as auto_sf
auto_sf = StatsForecast(models=auto_models, freq='D', n_jobs=-1)

In [40]:
df_stock_simple.head()

,unique_id,ds,y,y_origin
0,1019979|P,2022-12-09,8.570924,5275.0
1,1033235|P,2022-12-09,7.798523,2436.0
2,1093592|P,2022-12-09,5.049856,155.0
3,1113134|P,2022-12-09,8.806574,6677.0
4,1119767|P,2022-12-09,5.056246,156.0


In [41]:
# Check if the automatic StatsForecast model for sales is already saved
if 'stock_automatic_statsforecast_model.joblib' in saved_models:
    # Load the saved automatic StatsForecast model for sales
    auto_sf_stock = joblib.load(path_models + 'stock_automatic_statsforecast_model.joblib')
else:
    # Generate forecasts for the next horizon days
    auto_sf_stock = auto_sf.fit(df=df_stock_simple.drop(columns=['y_origin']))

    # Save the automatic StatsForecast model
    StatsForecast.save(auto_sf_stock, path_models + 'stock_automatic_statsforecast_model.joblib')

In [42]:
if 'stock_pred.joblib' in saved_models:
    # Load the saved stock predictions
    stock_pred_auto = joblib.load(path_models + 'stock_pred.joblib')
else:
    # Predict using automatic models
    stock_pred_auto = auto_sf_stock.predict(h=horizon)
    # Save the stock predictions
    StatsForecast.save(stock_pred_auto, path_models + 'stock_pred.joblib')

In [43]:
stock_pred_auto.head()

,unique_id,ds,AutoARIMA,AutoETS,CES,AutoTheta
0,1003530|P,2025-05-10,6.057827,6.057229,6.105691,6.004232
1,1003530|P,2025-05-11,6.057827,6.057229,6.064883,6.004009
2,1003530|P,2025-05-12,6.057827,6.057229,5.988057,6.003787
3,1003530|P,2025-05-13,6.057827,6.057229,6.093974,6.003564
4,1003530|P,2025-05-14,6.057827,6.057229,6.253950,6.003341


In [44]:
stock_pred_auto[ stock_pred_auto.select_dtypes(include=[np.number]).columns.to_list() ] = np.expm1( stock_pred_auto.select_dtypes(include=[np.number]) )
stock_pred_auto.head()

,unique_id,ds,AutoARIMA,AutoETS,CES,AutoTheta
0,1003530|P,2025-05-10,426.445799,426.189877,447.402560,404.139811
1,1003530|P,2025-05-11,426.445799,426.189877,429.472502,404.049582
2,1003530|P,2025-05-12,426.445799,426.189877,397.639284,403.959373
3,1003530|P,2025-05-13,426.445799,426.189877,442.179071,403.869185
4,1003530|P,2025-05-14,426.445799,426.189877,519.063049,403.779016


In [45]:
# StatsForecast plot
auto_sf.plot(plot_df, stock_pred_auto, engine='plotly')

#### Summary: Advanced Automated Forecasting Models

##### Objective
Evaluate the performance of advanced automated forecasting models — **AutoARIMA**, **AutoETS**, **AutoCES**, and **AutoTheta** — on daily time series with weekly seasonality (`season_length=7`) and a 28-day forecast horizon.


##### Key Observations

1. **Model Behavior:**
   - All models produced **smoothed forecasts**, often **flat or trendless**, contrasting with the highly **volatile and noisy** historical series.
   - Some series showed **constant-level forecasts** (e.g., `1147662|P`), suggesting the model interpreted noise as random and opted for a stable average.
   - Others displayed **subtle seasonal patterns** (e.g., `1992802|P`), indicating successful detection of weekly seasonality, likely by **AutoETS** or **AutoCES**.

2. **Technical Conclusions:**
   - **Noise vs. Structure:** Auto models captured underlying structure (level and weak seasonality), but struggled with extreme fluctuations, treating them as **unpredictable noise**.
   - **Model Robustness:** **AutoTheta** and **AutoETS** likely performed best due to their resilience to noise. **AutoARIMA** may have overfit volatile patterns.
   - **Need for Exogenous Variables:** To improve forecast accuracy and confidence, it's essential to incorporate **external features** (e.g., holidays, promotions, weather, commodity prices).

##### Recommendation

While Auto models provided stable forecasts and detected some seasonality, the **high noise level** in the data limits their precision. The next step is to explore models that support **external features**, such as **AutoARIMAX** or **machine learning approaches** like **LightGBM**.

---

# Sales Volume

In [46]:
df.head(3)

,received_date,product_id,sales_volume,stock_quantity,min_stock,supplier_id,delivery_lag,lead_time,max_stock,supplier_rating,maximum_days_on_sale,shelf_life_days,moq,distance_km,doi_inventory_turnover,inventory_turnover_rate,weather_severity,lpo,category,supplier,storage_recommendation,is_holiday,sales_demand,sub_category,expiration_status,wind_classification,precipitation_classification,unit_of_measurement,in_season,temperature_classification,day_classification,product,is_weekend,stock_log,sales_log
0,2022-12-09,1019979|P,526,5275,3520,1536622|S,10,7,4224,5.0,7.0,21.0,35.0,190.0,47.0,21.494078,Moderate,2022-11-29,Fresh Foods,Lemon & Lime Co.,Refrigerated,False,Normal,Fruits,Nearing,Gentle to Fresh Breeze,No precipitation,lb,True,Warm,Weekdays,Lime,False,8.570924,6.267201
1,2022-12-09,1033235|P,688,2436,2271,1644445|S,8,3,3028,3.0,2.0,5.0,75.0,45.0,28.0,35.607450,Moderate,2022-12-01,Bakery,Bakery Fresh Co.,Room Temperature,False,Normal,Bread,Expired,Gentle to Fresh Breeze,No precipitation,unit,False,Warm,Weekdays,White Bread,False,7.798523,6.535241
2,2022-12-09,1093592|P,33,155,155,1168079|S,2,5,186,2.0,1.0,2.0,40.0,180.0,44.0,22.882402,Moderate,2022-12-07,Fresh Foods,OceanHarvest Seafood,Refrigerated,False,Normal,Seafood,Nearing,Gentle to Fresh Breeze,No precipitation,lb,False,Warm,Weekdays,Sardines,False,5.049856,3.526361


In [47]:
df_sales = df.rename(columns={'received_date': 'ds', 'product_id': 'unique_id', 'sales_log': 'y'}).copy()

df_sales_simple = df_sales[['unique_id', 'ds', 'y', 'sales_volume']].copy()
df_sales_simple.head()

,unique_id,ds,y,sales_volume
0,1019979|P,2022-12-09,6.267201,526
1,1033235|P,2022-12-09,6.535241,688
2,1093592|P,2022-12-09,3.526361,33
3,1113134|P,2022-12-09,6.577861,718
4,1119767|P,2022-12-09,3.828641,45


In [48]:
if 'sales_multiple_models.joblib' in saved_models:
    # Load the saved multiple models StatsForecast for sales
    sales_forecasts_df = joblib.load(path_models + 'sales_multiple_models.joblib')
else:
    # Generate forecasts for the next 28 days with 95% prediction intervals
    sales_forecasts_df = sf.forecast(df=df_sales_simple.drop(columns=['sales_volume']), h=28, level=[90])

    # Save the multiple models StatsForecast for sales
    StatsForecast.save(sales_forecasts_df, path_models + 'sales_multiple_models.joblib')

In [49]:
# Inverse transformation of log1p to original scale using expm1
sales_forecasts_df[sales_forecasts_df.select_dtypes(include=[np.number]).columns.to_list()] = np.expm1(sales_forecasts_df.select_dtypes(include=[np.number]))

In [50]:
# Display the first 14 rows of the forecasts DataFrame
sales_forecasts_df.head(14)

,unique_id,ds,HoltWinters,HoltWinters-lo-90,HoltWinters-hi-90,SeasonalNaive,SeasonalNaive-lo-90,SeasonalNaive-hi-90,DynamicOptimizedTheta,DynamicOptimizedTheta-lo-90,DynamicOptimizedTheta-hi-90
0,1003530|P,2025-05-10,142.037986,59.039945,339.770891,209.0,59.879181,723.385564,115.926544,50.881126,324.864684
1,1003530|P,2025-05-11,151.934338,63.193914,363.347809,41.0,11.175836,143.877113,115.926544,44.856270,251.258474
2,1003530|P,2025-05-12,118.067926,48.978540,282.665173,104.0,29.439591,361.192782,115.926544,37.128028,274.681942
3,1003530|P,2025-05-13,107.264101,44.443643,256.926403,104.0,29.439591,361.192782,115.926544,45.112375,250.757703
4,1003530|P,2025-05-14,122.051579,50.650643,292.155908,96.0,27.120384,333.597142,115.926544,52.462968,303.005983
5,1003530|P,2025-05-15,146.155103,60.768023,349.579850,138.0,39.296220,478.474254,115.926544,48.729118,283.739664
6,1003530|P,2025-05-16,126.548273,52.538075,302.869012,93.0,26.250681,323.248776,115.926544,44.070439,297.230891
7,1003530|P,2025-05-17,142.507438,59.236861,340.890071,209.0,35.452327,1208.799314,115.926544,45.551817,300.523391
8,1003530|P,2025-05-18,152.436270,63.404419,364.544626,41.0,6.290465,240.959863,115.926544,43.784979,297.562533
9,1003530|P,2025-05-19,118.458709,49.142398,283.597139,104.0,17.226163,603.899657,115.926544,47.772339,264.566685


In [51]:
# Filter the DataFrame for dates after June 1, 2024
plot_df_sales = df_sales_simple.loc[df_sales_simple['ds'] > '2024-06-01'].copy()
plot_df_sales.drop(columns=['y'], inplace=True)
plot_df_sales.rename(columns={'sales_volume': 'y'}, inplace=True)

In [52]:
# Plot forecasts for the specified date range
sf.plot(plot_df_sales, sales_forecasts_df, engine='plotly')

## Evaluate the model’s performance

In [53]:
df_sales_simple.head()

,unique_id,ds,y,sales_volume
0,1019979|P,2022-12-09,6.267201,526
1,1033235|P,2022-12-09,6.535241,688
2,1093592|P,2022-12-09,3.526361,33
3,1113134|P,2022-12-09,6.577861,718
4,1119767|P,2022-12-09,3.828641,45


In [54]:
if 'sales_cv.joblib' in saved_models:
    # Load the saved cross-validation DataFrame for sales
    sales_cv_df = joblib.load(path_models + 'sales_cv.joblib')
else:
    # Perform cross-validation to evaluate model performance
    sales_cv_df = sf.cross_validation(
        df=df_sales_simple.drop(columns=['sales_volume']),
        h=28,
        step_size=28,
        n_windows=2
    )
    # Save the cross-validation DataFrame for sales
    StatsForecast.save(sales_cv_df, path_models + 'sales_cv.joblib')

In [55]:
# Display the first rows of the cross-validation DataFrame
sales_cv_df.head()

,unique_id,ds,cutoff,y,HoltWinters,SeasonalNaive,DynamicOptimizedTheta
0,1003530|P,2025-01-01,2024-12-30,5.525453,4.980040,4.394449,4.933411
1,1003530|P,2025-01-04,2024-12-30,5.998937,5.098044,4.304065,4.933411
2,1003530|P,2025-01-06,2024-12-30,4.859812,4.856770,5.068904,4.933411
3,1003530|P,2025-01-09,2024-12-30,5.313206,4.772030,4.875197,4.933411
4,1003530|P,2025-01-10,2024-12-30,4.644391,4.866963,4.532599,4.933411


In [56]:
# Evaluate model performance using Mean Absolute Error (MAE)
evaluation_mae_sales = evaluate_cv(sales_cv_df, mae)
evaluation_mae_sales.head()

,unique_id,HoltWinters,SeasonalNaive,DynamicOptimizedTheta,best_model
0,1003530|P,0.438826,0.522640,0.405064,DynamicOptimizedTheta
1,1007004|P,0.534624,0.877343,0.576124,HoltWinters
2,1009699|P,0.385232,0.555524,0.387958,HoltWinters
3,1017723|P,0.399271,0.551126,0.387007,DynamicOptimizedTheta
4,1018159|P,0.529221,0.620099,0.521121,DynamicOptimizedTheta


In [57]:
# Evaluate model performance using Root Mean Square Error (RMSE)
evaluation_rmse_sales = evaluate_cv(sales_cv_df, rmse)
evaluation_rmse_sales.head()

,unique_id,HoltWinters,SeasonalNaive,DynamicOptimizedTheta,best_model
0,1003530|P,0.528708,0.632792,0.495965,DynamicOptimizedTheta
1,1007004|P,0.669903,1.081688,0.707261,HoltWinters
2,1009699|P,0.479943,0.693043,0.490548,HoltWinters
3,1017723|P,0.468233,0.656639,0.469480,HoltWinters
4,1018159|P,0.665717,0.780191,0.645918,DynamicOptimizedTheta


In [58]:
# Display the best model counts based on MAE evaluation
evaluation_mae_sales['best_model'].value_counts().to_frame().reset_index()

,best_model,count
0,DynamicOptimizedTheta,90
1,HoltWinters,79
2,SeasonalNaive,1


In [59]:
# Display the best model counts based on RMSE evaluation
evaluation_rmse_sales['best_model'].value_counts().to_frame().reset_index()

,best_model,count
0,DynamicOptimizedTheta,87
1,HoltWinters,83


In [60]:
seasonal_ids = evaluation_rmse_sales.query('best_model == "DynamicOptimizedTheta"')['unique_id']
sf.plot(plot_df_sales, sales_forecasts_df, unique_ids=seasonal_ids, models=["HoltWinters","DynamicOptimizedTheta"], engine='plotly')

## Select the Best Model

In [61]:
# Prepare data for plotting
prod_sales_fc_df = get_best_model_forecast(sales_forecasts_df, evaluation_mae_sales)
prod_sales_fc_df.head()

,unique_id,ds,best_model,best_model-lo-90,best_model-hi-90
0,1003530|P,2025-05-10,115.926544,50.881126,324.864684
1,1003530|P,2025-05-11,115.926544,44.856270,251.258474
2,1003530|P,2025-05-12,115.926544,37.128028,274.681942
3,1003530|P,2025-05-13,115.926544,45.112375,250.757703
4,1003530|P,2025-05-14,115.926544,52.462968,303.005983


In [62]:
# Plot to unique_ids and some selected models
sf.plot(plot_df_sales, prod_sales_fc_df, level=[90], engine='plotly')


## Detailed Graphical Analysis of Sales Volume

The charts compare historical data (`y`) with forecasts from **HoltWinters (HW)**, **SeasonalNaive (SN)**, **DynamicOptimizedTheta (DOT)**, and the **Best Model** including its confidence interval.

### 1. Historical Series Characteristics

- **Extreme Volatility:** All series (dark blue line) show high data dispersion and sharp peaks, indicating significant noise and forecasting difficulty.
- **Stable Trend:** No clear long-term upward or downward trend is visible, suggesting the series fluctuates around a stable average level.

### 2. Behavior of Base Models

- **HoltWinters (HW) – Red/Purple Line:**
  - HW is the most dynamic model, capturing a clear **7-day seasonal pattern**, reflecting weekly sales variations (e.g., weekends).
- **DynamicOptimizedTheta (DOT) – Yellow/Orange Line:**
  - DOT produces **flat and conservative forecasts**, essentially projecting the historical average with minimal seasonal variation.
- **Key Insight:** HW’s ability to capture seasonality, unlike DOT, aligns with previous metric results (HW excelled in MAE and RMSE), confirming **seasonality as the dominant structural component**.

### 3. Uncertainty Analysis

- **Best Model Forecast (Purple Line):** Often appears **smoothed and nearly constant**, even when HW detects seasonality—due to conservative averaging.
- **Confidence Interval (`level_90` – Purple Shaded Area):** Extremely **wide and persistent** across all series. For example, a forecast around 400 units may span from 0 to 800.
- **Implication:** The wide interval visually confirms **high uncertainty and unexplained variance**, meaning actual values may differ significantly due to historical noise.


### Final Conclusion

The sales volume analysis reveals:

1. **Preferred Model:** **HoltWinters** is the most suitable time series model, effectively capturing weekly seasonality.
2. **Critical Limitation:** All univariate models (based solely on past sales) hit a precision ceiling due to unpredictable spikes and noise, resulting in **high uncertainty**.

#### Recommendation for Next Phase

Shift focus from purely historical models (`y`) to those incorporating **external drivers of peaks and troughs** (e.g., promotions, holidays, marketing campaigns).

**Next Step:** Implement models with exogenous variables (`X`), such as:
- **AutoARIMAX**
- **Machine Learning models** (e.g., LightGBM, XGBoost) using calendar features and business variables.

---

##  Automatic Models for Sales Volume

In [63]:
df_sales_simple.head()

,unique_id,ds,y,sales_volume
0,1019979|P,2022-12-09,6.267201,526
1,1033235|P,2022-12-09,6.535241,688
2,1093592|P,2022-12-09,3.526361,33
3,1113134|P,2022-12-09,6.577861,718
4,1119767|P,2022-12-09,3.828641,45


In [64]:
# Check if the automatic StatsForecast model for sales is already saved
if 'sales_automatic_statsforecast_model.joblib' in saved_models:
    # Load the saved automatic StatsForecast model for sales
    auto_sf_sales = joblib.load(path_models + 'sales_automatic_statsforecast_model.joblib')
else:
    # Generate forecasts for the next horizon days
    auto_sf_sales = auto_sf.fit(df=df_sales_simple.drop(columns=['sales_volume']))

    # Save the automatic StatsForecast model
    StatsForecast.save(auto_sf_sales, path_models + 'sales_automatic_statsforecast_model.joblib')

In [65]:
# Check if the sales predictions are already saved
if 'sales_predictions.joblib' in saved_models:
    # Load the saved sales predictions
    sales_pred_auto = joblib.load(path_models + 'sales_predictions.joblib')
else:
    # Predict using automatic modelspred_auto
    sales_pred_auto = auto_sf_sales.predict(h=horizon)

    # Save the sales predictions
    StatsForecast.save(sales_pred_auto, path_models + 'sales_predictions.joblib')

In [66]:
sales_pred_auto.head()

,unique_id,ds,AutoARIMA,AutoETS,CES,AutoTheta
0,1003530|P,2025-05-10,4.837357,4.83704,4.988376,4.761641
1,1003530|P,2025-05-11,4.837357,4.83704,5.034067,4.761641
2,1003530|P,2025-05-12,4.837357,4.83704,4.848280,4.761641
3,1003530|P,2025-05-13,4.837357,4.83704,4.741216,4.761641
4,1003530|P,2025-05-14,4.837357,4.83704,4.864909,4.761641


In [67]:
sales_pred_auto[ sales_pred_auto.select_dtypes(include=[np.number]).columns.to_list() ] = np.expm1( sales_pred_auto.select_dtypes(include=[np.number]) )
sales_pred_auto.head()

,unique_id,ds,AutoARIMA,AutoETS,CES,AutoTheta
0,1003530|P,2025-05-10,125.135522,125.095555,145.697927,115.93762
1,1003530|P,2025-05-11,125.135522,125.095555,152.556215,115.93762
2,1003530|P,2025-05-12,125.135522,125.095555,126.520903,115.93762
3,1003530|P,2025-05-13,125.135522,125.095555,113.573473,115.93762
4,1003530|P,2025-05-14,125.135522,125.095555,128.659103,115.93762


In [68]:
# StatsForecast plot
sf.plot(plot_df_sales, sales_pred_auto, engine='plotly')


## Analysis of Auto Models Sales Volume Forecasting

The charts compare historical sales series (`y`) with forecasts from:

- **AutoARIMA**
- **AutoETS**
- **AutoCES**
- **AutoTheta**

### 1. Historical Series Characteristics

- **Extreme Noise:** The dark blue line (`y`) is highly volatile, with frequent sharp peaks and drops.
- **No Strong Trend:** Most series fluctuate around a stable average, with no clear long-term trend.

### 2. Behavior of Auto Models

These models automatically select the best structure (ARIMA, ETS, CES, Theta) and show cautious behavior in noisy environments:

- **Overly Smoothed Forecasts:** For nearly all `unique_id`s, the forecast lines (red, yellow, pink, brown) are **flat and closely aligned**, converging to the series average and ignoring most historical volatility.
  - **Subtle Seasonality Exception:** In a few cases (e.g., `unique_id=1326719|P`), a faint repetitive pattern may appear, but it's far less pronounced than the 7-day seasonality captured by HoltWinters.
- **Model Consistency:** Despite using different methodologies, all four Auto models produce nearly identical forecasts, indicating:
  1. **Low signal-to-noise ratio**
  2. **Weak seasonality and trend components**
  3. **The optimal forecast is simply the series average**

### 3. Reinforced Conclusion

This new analysis confirms previous findings:

- **Main Challenge:** Forecasting sales volume is difficult due to **high variance and unexplained noise**.
- **Univariate Model Limitation:** Models relying solely on historical data (`y`) fail to predict extreme events, resulting in conservative forecasts.
- **Next Step:** To significantly improve accuracy, it's **essential** to include **exogenous variables (X)** that explain volatility, such as:
  - **Calendar Factors:** Holidays, weekdays, workdays
  - **Business Factors:** Promotions, price changes, marketing events

The logical next move is to implement **AutoARIMAX** or, preferably, **Machine Learning regression models** (e.g., LightGBM, XGBoost), which are better suited to handle multiple features and noise.